<a href="https://colab.research.google.com/github/XescLlabres/tfgFLM/blob/main/HUSE_per_pacient/KNNImputer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Taula amb dades huse

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data1 = pd.read_csv('df_knnimputer.csv')

In [ ]:
data1.head()

,ID,CLAVE,AGE_AT_TACE,DAYS_CT_PRETACE,WEIGHT,HEIGHT,BMI,BMI_CATEGORY,ALBI_Score,Meld_Original,...,Tumoral_Distribution,I,II,III,IVa,IVb,V,VI,VII,VIII
0,222.0,222.0,82.0,47.0,64.0,1.47,29.62,3.0,-2.12,6.43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,346.0,346.0,71.0,26.0,55.0,1.52,23.81,2.0,-1.61,8.49,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,613.0,613.0,48.0,75.0,65.0,1.69,22.76,2.0,-1.58,14.73,...,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
3,697.0,697.0,77.0,110.0,68.0,1.59,26.90,3.0,-2.12,9.02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,745.0,745.0,54.0,28.0,65.0,1.70,22.49,2.0,-2.89,7.19,...,2.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0


In [ ]:
data1.drop(['ID', 'CLAVE'], axis=1)

,AGE_AT_TACE,DAYS_CT_PRETACE,WEIGHT,HEIGHT,BMI,BMI_CATEGORY,ALBI_Score,Meld_Original,Meld-Na,Globulos_blancos,...,Tumoral_Distribution,I,II,III,IVa,IVb,V,VI,VII,VIII
0,82.0,47.0,64.0,1.47,29.62,3.0,-2.12,6.43,6.43,9.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,71.0,26.0,55.0,1.52,23.81,2.0,-1.61,8.49,8.49,5.05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,48.0,75.0,65.0,1.69,22.76,2.0,-1.58,14.73,16.63,4.72,...,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
3,77.0,110.0,68.0,1.59,26.90,3.0,-2.12,9.02,9.02,4.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,54.0,28.0,65.0,1.70,22.49,2.0,-2.89,7.19,7.19,5.85,...,2.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,83.0,194.0,75.0,1.60,29.30,3.0,-2.60,8.69,9.79,5.39,...,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
127,67.0,578.0,94.0,1.74,31.05,4.0,-2.83,9.11,9.11,3.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
128,67.0,52.0,65.0,1.74,21.47,2.0,-1.49,9.82,9.82,9.72,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
129,74.0,269.0,83.0,1.65,30.49,4.0,-2.73,6.65,6.65,6.26,...,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0


In [ ]:
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut

data1.columns = data1.columns.str.strip()

y = data1.loc[:, "RECIST"].values

X = data1.loc[:, (data1.columns != "RECIST") & (data1.columns != "ID")].values

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)


loo = LeaveOneOut()
model = LogisticRegression(max_iter=1000)
all_bootstrap_probabilities = []

for train_index, test_index in loo.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    bootstrapped_probabilities = []

    for _ in range(100):
        X_train_bootstrap, y_train_bootstrap = resample(X_train, y_train, replace=True)

        model.fit(X_train_bootstrap, y_train_bootstrap)

        prob = model.predict_proba(X_test)[:, 1][0]
        bootstrapped_probabilities.append(prob)

    all_bootstrap_probabilities.append(bootstrapped_probabilities)

df_probabilities = pd.DataFrame(all_bootstrap_probabilities)

df_probabilities.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.998269,0.879119,0.998106,0.964410,0.499279,0.994373,0.993746,0.969376,0.999963,0.333040,...,0.987748,0.864757,0.940614,0.956725,0.996676,0.937723,0.965905,0.992444,0.999873,0.990000
1,0.994570,0.852385,0.991596,0.993660,0.991343,0.741944,0.973867,0.986017,0.949345,0.988177,...,0.993577,0.983196,0.994836,0.992018,0.928230,0.806758,0.960725,0.997307,0.999601,0.868845
2,0.897040,0.495375,0.899528,0.994943,0.914163,0.493087,0.964092,0.295715,0.618833,0.842527,...,0.735523,0.824685,0.853340,0.604309,0.131809,0.416922,0.855460,0.998734,0.921421,0.724551
3,0.872167,0.991867,0.805905,0.983859,0.946621,0.779487,0.971054,0.976475,0.998552,0.941080,...,0.986508,0.991266,0.746382,0.938779,0.913205,0.891197,0.986415,0.766567,0.950149,0.941838
4,0.999861,0.999123,0.999576,0.999944,0.998060,0.993383,0.922457,0.934107,0.997705,0.999887,...,0.981556,0.945567,0.999987,0.999013,0.934135,0.974247,0.994126,0.999880,0.999134,0.999119


In [ ]:

df_probabilities = pd.DataFrame(all_bootstrap_probabilities)

data1 = pd.read_csv('df_knnimputer.csv')
data1.drop(['CLAVE'],axis=1,inplace=True)
df_probabilities.insert(0, 'ID', data1['ID'].values)
df_probabilities.to_csv("huse2bootstrap_probabilities_id.csv", index=False)



print("Archivo guardado con el identificador 'id' en la primera columna.")

Archivo guardado con el identificador 'id' en la primera columna.


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

In [ ]:
taula1 = pd.read_csv('huse2bootstrap_probabilities_id.csv')
taula2 = pd.read_csv('df_knnimputer.csv')

taula2.drop(['CLAVE'],axis=1,inplace=True)

In [ ]:

id_column = taula1.iloc[:, 0]

probabilities = taula1.iloc[:, 1:]

y_pred = (probabilities.mean(axis=1) >= 0.5).astype(int)

taula1_processed = pd.DataFrame({'ID': id_column, 'RECIST': y_pred})
taula2_processed = taula2[['ID', 'RECIST']]
taula2_processed.rename(columns={'RECIST': 'y_true'}, inplace=True)


taula05 = pd.merge(taula1_processed, taula2_processed, on='ID')

print(taula05.head())

      ID  RECIST  y_true
0  222.0       1     0.0
1  346.0       1     0.0
2  613.0       1     1.0
3  697.0       1     1.0
4  745.0       1     0.0


<ipython-input-9-735a19bb8885>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taula2_processed.rename(columns={'RECIST': 'y_true'}, inplace=True)


In [ ]:
y_pred = taula05['RECIST']
y_true = taula05['y_true']

cm = confusion_matrix(y_true, y_pred)
print("Matriu de confusió:")
print(cm)

precision = precision_score(y_true, y_pred)
print(f"Precisió: {precision:.2f}")

recall = recall_score(y_true, y_pred)
print(f"Recall: {recall:.2f}")

f1 = f1_score(y_true, y_pred)
print(f"F1-Score: {f1:.2f}")

accuracy = accuracy_score(y_true, y_pred)
print(f"Exactitut: {accuracy:.2f}")

Matriz de Confusión:
[[  0  21]
 [  8 102]]
Precisión: 0.83
Recall: 0.93
F1-Score: 0.88
Exactitud: 0.78


In [ ]:
df_data = pd.read_csv("df_knnimputer.csv")
df_data.drop(['CLAVE'],axis=1,inplace=True)

df_probabilities = pd.read_csv("huse2bootstrap_probabilities_id.csv")

df_data_subset = df_data[['ID', 'RECIST']]

if len(df_data_subset) == len(df_probabilities):
    df_final = pd.concat([df_data_subset, df_probabilities], axis=1)

    df_final.to_csv("final_table.csv", index=False)

    print("Tabla final creada y guardada en 'final_table.csv'.")
else:
    print("Error: Los DataFrames tienen diferente número de filas. Verifica tus datos.")

Tabla final creada y guardada en 'final_table.csv'.


In [ ]:
df = pd.read_csv('final_table.csv')

id_column = 'ID'
diagnosis_column = 'RECIST'
prob_columns = df.columns[3:]

def calcular_intervalo(probabilidades, li, ls):
    lower_bound = np.percentile(probabilidades, li)
    upper_bound = np.percentile(probabilidades, ls)
    return lower_bound, upper_bound

In [ ]:
intervalos_confianza = []

for index, row in df.iterrows():
    probabilidades = row[prob_columns].values
    lower, upper = calcular_intervalo(probabilidades, 2.5, 97.5)
    intervalos_confianza.append([row[id_column], row[diagnosis_column], lower, upper])

df_intervalos = pd.DataFrame(intervalos_confianza, columns=[id_column, diagnosis_column, 'lower_bound', 'upper_bound'])

In [ ]:
def calcular_TP(a, b):
    TP = min(1 - a, b)
    return TP
def calcular_AP(a,b):
    AP = min(a, 1-b)
    return AP
def calcular_EP(a,b):
    EP = (b-a)
    return EP

In [ ]:
df_intervalos['TP'] = df_intervalos.apply(lambda row: calcular_TP(row['lower_bound'], row['upper_bound']), axis=1)

df_intervalos['AP'] = df_intervalos.apply(lambda row: calcular_AP(row['lower_bound'], row['upper_bound']), axis=1)

df_intervalos['EP'] = df_intervalos.apply(lambda row: calcular_EP(row['lower_bound'], row['upper_bound']), axis=1)

In [ ]:
medias = df_intervalos[['TP', 'AP', 'EP']].mean()
print("Medias:")
print(medias)

Medias:
TP    0.580955
AP    0.001086
EP    0.579870
dtype: float64
